In [ ]:
import dice_ml
import warnings
import pandas as pd


def visualize(
    X,  # les données
    # y=None,
    clf,
    backend="sklearn",
    n=10,
    eta=2,
    n_gs=100,
    ax=None,
):
    def plot_boundaries(X, y, ax, clf):
        """ "Plot the data and the decision boundary resulting from a classifier."""
        x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
        y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
        DecisionBoundaryDisplay.from_estimator(clf, X, ax=ax, eps=0.5)
        # Plot the training points
        ax.scatter(X[:, 0], X[:, 1], c=y, edgecolors="k")
        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)
        ax.set_xticks(())
        ax.set_yticks(())

    # if not clf and not y:
    #     y = clf.fit_predict(X)
    y = clf.fit_predict(X)

    df = pd.DataFrame(X, columns=["X", "Y"])
    df["Target"] = y

    data = dice_ml.Data(
        dataframe=df,
        continuous_features=["X", "Y"],
        outcome_name="Target",
    )
    model = dice_ml.Model(model=clf, backend=backend)
    if backend == "sklearn":
        explainer = dice_ml.Dice(data, model, method="random")
    else:
        explainer = dice_ml.Dice(data, model)

    warnings.simplefilter("ignore")
    counterfactuals = explainer.generate_counterfactuals(
        query_instances=df.loc[0:1, ["X", "Y"]],
        total_CFs=4,
        desired_class="opposite",
        proximity_weight=0.5,
        diversity_weight=1.0,
        features_to_vary="all",
        permitted_range=None,
        posthoc_sparsity_param=0.1,
    )

    if not ax:
        _, ax = plt.subplots()
    plot_boundaries(X, y, ax, clf)
    first_outliers = np.argwhere(y == 1)[0]
    ax.scatter(
        df.loc[first_outliers, "X"],
        df.loc[first_outliers, "Y"],
        c="red",
        marker="o",
        label="Obs",
        s=100,
    )
    enemy_df = counterfactuals.cf_examples_list[0].final_cfs_df
    ax.scatter(enemy_df["Feature1"], enemy_df["Feature2"], c="blue", marker="x", s=100)

    return enemy_df


class CustomIsolationForest(IsolationForest):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def predict_proba(self, X):
        return super().score_samples(X)


clf = IsolationForest(contamination=outliers_fraction, random_state=42)
clf.predict_proba = lambda x: clf.score_samples(x)
visualize(X_similar, clf=clf)
# for name, clf in anomaly_algorithms:
# fig = plt.figure(figsize=(27, 6))
# for i, (dataset, noise) in enumerate(zip(datasets, noise_lvl)):
#     X, y = dataset
#     clf = clf.fit(X, y)
#     ax = fig.add_subplot(1, len(datasets), i + 1)
#     ax.set_title(name)
#     visualize(X, y, clf, "sklearn", ax=ax)
# fig.tight_layout()